In [1]:
import yfinance as yf
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import LSTM, Dense, Dropout, Input
from keras._tf_keras.keras.regularizers import L2

In [2]:
def format_data(stock):
    stock_chosen = yf.Ticker(f"{stock}")
    stock_data = stock_chosen.history(start='2010-01-01', end='2024-09-01', interval="1d")
    
    stock_data.dropna(inplace=True)
    sp500_data = yf.download("^GSPC", start='2010-01-01', end='2024-09-01', interval="1d")
    sp500_data = sp500_data['Close'].rename('S&P500_Close')
    stock_data.index = stock_data.index.tz_localize(None)
    sp500_data.index = sp500_data.index.tz_localize(None)
    stock_data = stock_data.join(sp500_data, how='inner')
    
    print(stock_data.columns)
    features = ['Open', 'Close', 'Volume', 'S&P500_Close']
    stock_data = stock_data[features].dropna()
    
    stock_data['Target'] = stock_data['Close'].shift(-1)
    stock_data.dropna(inplace=True)
    
    return stock_data

Choosing a stock to train our model on: 

In [3]:
features = ['Open', 'Close', 'Volume', 'S&P500_Close']
stock_data = format_data("GOOG")

[*********************100%***********************]  1 of 1 completed

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'S&P500_Close'],
      dtype='object')


In [4]:
X = stock_data[features]  
Y = stock_data['Target']  


Creating training and testing data:

In [5]:
def create_train_test(stock_data, features, test_size=0.2, val_size=0.2):
    X = stock_data[features]
    Y = stock_data['Target']
    
    # Separate test set
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    
    # Separate validation set from the remaining data
    val_ratio = val_size / (1 - test_size)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=val_ratio, random_state=42)
    
    return X_train, X_val, X_test, Y_train, Y_val, Y_test


In [7]:
X_train, X_val, X_test, Y_train, Y_val, Y_test = create_train_test(stock_data, features)


In [8]:
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)
print(X_test.shape, Y_test.shape)


(2213, 4) (2213,)
(738, 4) (738,)
(738, 4) (738,)


In [9]:
print(stock_data.index[-len(Y_test):])  # Ensure this matches with test set plotting


DatetimeIndex(['2021-09-23', '2021-09-24', '2021-09-27', '2021-09-28',
               '2021-09-29', '2021-09-30', '2021-10-01', '2021-10-04',
               '2021-10-05', '2021-10-06',
               ...
               '2024-08-16', '2024-08-19', '2024-08-20', '2024-08-21',
               '2024-08-22', '2024-08-23', '2024-08-26', '2024-08-27',
               '2024-08-28', '2024-08-29'],
              dtype='datetime64[ns]', name='Date', length=738, freq=None)


Now, we implement a MinMaxScaler to scale our data and ensure that all values are between 0 and 1:


In [10]:
def createScalar(sequence_length=60):
    # Initialize scalers
    scaler_X = MinMaxScaler()
    scaler_Y = MinMaxScaler()
    
    # Scale X data
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)
    X_test_scaled = scaler_X.transform(X_test)
    
    # Scale Y values in 2D
    Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1))
    Y_val_scaled = scaler_Y.transform(Y_val.values.reshape(-1, 1))
    Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1))
    
    # Create sequences for LSTM
    X_train_seq, Y_train_seq = create_sequences(X_train_scaled, Y_train_scaled, sequence_length)
    X_val_seq, Y_val_seq = create_sequences(X_val_scaled, Y_val_scaled, sequence_length)
    X_test_seq, Y_test_seq = create_sequences(X_test_scaled, Y_test_scaled, sequence_length)
    
    return X_train_seq, X_val_seq, X_test_seq, Y_train_seq, Y_val_seq, Y_test_seq, scaler_Y

def create_sequences(X_data, Y_data, sequence_length=60):
    Xs, Ys = [], []
    for i in range(len(X_data) - sequence_length):
        Xs.append(X_data[i:(i + sequence_length)])
        Ys.append(Y_data[i + sequence_length])
    return np.array(Xs), np.array(Ys)


X_train_seq, X_val_seq, X_test_seq, Y_train_seq, Y_val_seq, Y_test_seq, scaler_Y = createScalar()

In [11]:
print(X_train_seq.shape)  # Expected shape: (samples, sequence_length, features)
print(Y_train_seq.shape)  # Expected shape: (samples, 1)

(2153, 60, 4)
(2153, 1)


In [ ]:
window_size = 120


In [ ]:
def createModel(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    return model

# Compile the model, train the model, and evaluate the model:

input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
model = createModel(input_shape)
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(
    X_train_seq, Y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_seq, Y_val_seq),
    verbose=1
)

In [ ]:
# Evaluate the model
train_loss = model.evaluate(X_train_seq, Y_train_seq    , verbose=0)
val_loss = model.evaluate(X_val_seq, Y_val_seq, verbose=0)
test_loss = model.evaluate(X_test_seq, Y_test_seq, verbose=0)

print(f"Train Loss: {train_loss}")
print(f"Validation Loss: {val_loss}")
print(f"Test Loss: {test_loss}")

# Make predictions
train_predictions = model.predict(X_train_seq)
val_predictions = model.predict(X_val_seq)
test_predictions = model.predict(X_test_seq)

# Calculate RMSE percentage error
def rmse_percentage_error(y_true, y_pred):
    y_true = scaler_Y.inverse_transform(y_true)
    y_pred = scaler_Y.inverse_transform(y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mean_price = np.mean(y_true)
    return (rmse / mean_price) * 100

train_rmse_pct = rmse_percentage_error(Y_train_seq, train_predictions)
val_rmse_pct = rmse_percentage_error(Y_val_seq, val_predictions)
test_rmse_pct = rmse_percentage_error(Y_test_seq, test_predictions)

print(f"Train RMSE Percentage Error: {train_rmse_pct:.2f}%")
print(f"Validation RMSE Percentage Error: {val_rmse_pct:.2f}%")
print(f"Test RMSE Percentage Error: {test_rmse_pct:.2f}%")

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(12,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Plot actual vs predicted values
def plot_predictions(y_true, y_pred, title):
    y_true = scaler_Y.inverse_transform(y_true).flatten()
    y_pred = scaler_Y.inverse_transform(y_pred).flatten()
    
    # Get the corresponding dates for the test set
    test_dates = stock_data.index[-len(y_true):]
    
    plt.figure(figsize=(16,8))
    plt.plot(test_dates, y_true, label='Actual', linewidth=2)
    plt.plot(test_dates, y_pred, label='Predicted', linewidth=2)
    plt.title(title, fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Stock Price', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    
    # Add text with min and max values
    plt.text(0.02, 0.98, f'Actual range: {y_true.min():.2f} - {y_true.max():.2f}', 
             transform=plt.gca().transAxes, verticalalignment='top')
    plt.text(0.02, 0.94, f'Predicted range: {y_pred.min():.2f} - {y_pred.max():.2f}', 
             transform=plt.gca().transAxes, verticalalignment='top')
    
    plt.tight_layout()
    plt.show()

plot_predictions(Y_test_seq, test_predictions, 'Test Set: Actual vs Predicted')

# Plot prediction error distribution
def plot_error_distribution(y_true, y_pred, title):
    y_true = scaler_Y.inverse_transform(y_true)
    y_pred = scaler_Y.inverse_transform(y_pred)
    errors = y_true - y_pred
    
    plt.figure(figsize=(12,6))
    sns.histplot(errors, kde=True)
    plt.title(title)
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    plt.show()

plot_error_distribution(Y_test_seq, test_predictions, 'Test Set: Prediction Error Distribution')

In [ ]:
def plot_stock_prediction(stock_data, y_true, y_pred, scaler_y, sequence_length):
    # Get the closing prices for the entire stock data
    closing_prices = stock_data['Close'].values.reshape(-1, 1)

    # Calculate the split point (80% of the data) to separate historical data from predictions
    split_point = int(len(closing_prices) * 0.8)

    # Create a time index from the stock data index
    time_index = stock_data.index

    # Inverse transform the predictions and actual values
    y_true_inv = scaler_y.inverse_transform(y_true)
    y_pred_inv = scaler_y.inverse_transform(y_pred)

    # The predicted data starts after the sequence length, so adjust the time index accordingly
    test_time_index = time_index[-len(y_true):]  # Time index for the test set

    # Plot the data
    plt.figure(figsize=(16, 8))

    # Plot the historical data (before the test set)
    plt.plot(time_index[:split_point], closing_prices[:split_point], label='Historical Data', color='blue')

    # Plot the actual values (from the test set)
    plt.plot(test_time_index, y_true_inv, label='Actual', color='green')

    # Plot the predicted values
    plt.plot(test_time_index, y_pred_inv, label='Predicted', color='red')

    # Add vertical line to separate historical and prediction data
    plt.axvline(x=time_index[split_point], color='gray', linestyle='--')

    plt.title('Stock Price: Historical Data, Actual vs Predicted')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.grid(True)
    plt.show()

# Call the function to create the plot (use the sequence-based test predictions and actual values)
plot_stock_prediction(stock_data, Y_test_seq, test_predictions, scaler_Y, sequence_length)
